### News

gemma3n_s1_botanist_20250721
this is stage 1 of botanist.
we first let the model see image and can specify the inflorescence , flower_arrangement, stem_pattern, flower_density, color ;
as much as possible ;
should need only one dict about inflorescence visual description because others are self explanatory.


output in JSON;

after the model can get image and return json of this.
we will let the model think that from all visuals,
the next stage will focus on using visual featues to confirm species name

todo;
read dataset mekpro/plantnet300k_grpo
and construct json
add dictionary inflorescence

Instruction, as a botanist, describe this flower image in JSON format.

answer
{
  "color" :
  "flower_arrangement" :    
  "stem_pattern" :        
  "flower_density" :    
  "inflorescence" :  
  "inflorescence_description" :    
  "species_name" :   
}

reward function :
- inflorescence_species consistency :  if consistent + score , if not - score
- color_species consistency :
- flower_arrangement_species :   

### Installation

In [1]:
'''
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth

# Install latest transformers for Gemma 3N
!pip install --no-deps --upgrade timm # Only for Gemma 3N
'''

'\nimport os\nif "COLAB_" not in "".join(os.environ.keys()):\n    !pip install unsloth\nelse:\n    # Do this only in Colab notebooks! Otherwise use pip install unsloth\n    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo\n    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer\n    !pip install --no-deps unsloth\n\n# Install latest transformers for Gemma 3N\n!pip install --no-deps --upgrade timm # Only for Gemma 3N\n'

In [2]:
!export HF_HUB_ENABLE_HF_TRANSFER=1
from google.colab import drive
import os

drive.mount('/content/drive')
output_dir = f"/content/drive/MyDrive/colab_output/gemma-3n-botanist9"
os.makedirs(output_dir, exist_ok=True)

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Unsloth

`FastModel` supports loading nearly any model now! This includes Vision and Text models!

In [3]:
from unsloth import FastModel
import torch
torch._dynamo.config.suppress_errors = True  # Optional: suppress if errors occur


model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3n-E4B-it-litert-preview",
    #model_name = "unsloth/gemma-3-4b-it",
    dtype = torch.bfloat16, # None for auto detection
    max_seq_length = 512, # Choose any for long context!
    load_in_4bit = False,  # 4 bit quantization to reduce memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    #use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.1: Fast Gemma3N patching. Transformers: 4.54.0.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gemma3N does not support SDPA - switching to eager!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

# Gemma 3N can process Text, Vision and Audio!

Let's first experience how Gemma 3N can handle multimodal inputs. We use Gemma 3N's recommended settings of `temperature = 1.0, top_p = 0.95, top_k = 64`

In [4]:
from transformers import TextStreamer
# Helper function for inference
def do_gemma_3n_inference(messages, max_new_tokens = 128):
    _ = model.generate(
        **tokenizer.apply_chat_template(
            messages,
            add_generation_prompt = True, # Must add for generation
            tokenize = True,
            return_dict = True,
            return_tensors = "pt",
        ).to("cuda"),
        max_new_tokens = max_new_tokens,
        temperature = 1.0, top_p = 0.95, top_k = 64,
        streamer = TextStreamer(tokenizer, skip_prompt = True),
    )



# Gemma 3N can see images!

<img src="https://raw.githubusercontent.com/plantnet/PlantNet-300K/refs/heads/main/images/1.jpg">

In [5]:
link = "https://raw.githubusercontent.com/plantnet/PlantNet-300K/refs/heads/main/images/1.jpg" # Cirsium arvense

messages = [{
    "role" : "user",
    "content": [
        { "type": "image", "image" : link },
        { "type": "text",  "text" : "As a botanist, specify inflorescence and speices name of this flower." }
    ]
}]

#do_gemma_3n_inference(messages, max_new_tokens = 50)

#print("broken after here")
#from unsloth.chat_templates import get_chat_template
#get_chat_template(tokenizer, chat_template = "gemma-3")
#do_gemma_3n_inference(messages, max_new_tokens = 50)


# Let's finetune Gemma 3N!

You can finetune the vision and text parts for now through selection - the audio part can also be finetuned - we're working to make it selectable as well!

We now add LoRA adapters so we only need to update a small amount of parameters!

In [6]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = True, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 256,           # Larger = higher accuracy, but might overfit
    lora_alpha = 256,  # Recommended alpha == r at least
    lora_dropout = 0.2,
    bias = "none",
    random_state = 3407,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.2.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


Unsloth: Making `model.base_model.model.model.language_model` require gradients


<a name="Data"></a>
### Data Prep
We now use the `Gemma-3` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. Gemma-3 renders multi turn conversations like below:

```
<bos><start_of_turn>user
Hello!<end_of_turn>
<start_of_turn>model
Hey there!<end_of_turn>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3, phi4, qwen2.5, gemma3` and more.

We get the first 3000 rows of the dataset

In [7]:
from datasets import load_dataset

full_train_dataset = load_dataset("mekpro/plantnet300k_observe", split="train[:30000]")
split_datasets = full_train_dataset.train_test_split(test_size=0.07, seed=42)
train_dataset = split_datasets['train']
val_dataset = split_datasets['test']  # 'test' here is actually our validation set

print(f"Train size: {len(train_dataset)}")
print(f"Validation size: {len(val_dataset)}")

Train size: 19754
Validation size: 1487


We now use `standardize_data_formats` to try converting datasets to the correct format for finetuning purposes!

In [8]:
import json

instruction = "As a botanist, observe the image of the flower and describe its visual features and species_name in JSON format. {color, inflorescencetype, inflorescence_description, flower_arrangement, flower_density, species, family, genus}"
def convert_to_conversation(sample):
     result = {
          "color" : sample["color"],
          "inflorescencetype"  : sample["inflorescencetype"],
          "inflorescence_description" : sample["inflorescence_description"],
          "flower_arrangement" : sample["flower_arrangement"],
          "flower_density" : sample["flower_density"],
#          "unique_visual_description" : sample ["unique_visual_description"],
#          "morphological_traits_observable_in_photograph": sample["morphological_traits_observable_in_photograph"],
          "species" : sample["species"],
          "family" : sample["family"],
          "genus" : sample["genus"],
 #         "visual_contrast_with_similar_species" : sample["visual_contrast_with_similar_species"]
     }
     conversation = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": sample["image"]},
                {"type": "text", "text": instruction},
            ],
        },
        {"role": "assistant", "content": [{"type": "text", "text": json.dumps(result) }]},
     ]
     return {"messages": conversation}
train_dataset_c = [convert_to_conversation(sample) for sample in train_dataset]
val_dataset_c = [convert_to_conversation(sample) for sample in val_dataset]
train_dataset_c[0]

{'messages': [{'role': 'user',
   'content': [{'type': 'image',
     'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=384x384>},
    {'type': 'text',
     'text': 'As a botanist, observe the image of the flower and describe its visual features and species_name in JSON format. {color, inflorescencetype, inflorescence_description, flower_arrangement, flower_density, species, family, genus}'}]},
  {'role': 'assistant',
   'content': [{'type': 'text',
     'text': '{"color": "Bicolor; white petals with a prominent, feathered pink \'eye\' or throat at the center.", "inflorescencetype": "Pseudo-umbel", "inflorescence_description": "A rounded flower head on an erect stalk, with the bicolored pattern creating a striking effect.", "flower_arrangement": "Individual flowers are clustered tightly, with the central color of each creating a stippled look.", "flower_density": "Dense", "species": "Pelargonium zonale", "family": "Geraniaceae", "genus": "Pelargonium"}'}]}]}

In [9]:
from torchvision import transforms
from PIL import Image
from unsloth.trainer import UnslothVisionDataCollator

plant_augmentations = transforms.Compose([
    transforms.RandomRotation(degrees=30),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1, hue=0.1),
    transforms.RandomResizedCrop(size=(384, 384), scale=(0.8, 1.0)),
])

# Custom collator with augmentation
class AugmentedUnslothVisionDataCollator(UnslothVisionDataCollator):
    def __init__(self, processor, image_processor, is_training=True):
        super().__init__(processor, image_processor)
        self.is_training = is_training

    def __call__(self, features):
        # Apply augmentation only if training flag is set
        if self.is_training:
            for feature in features:
                if 'image' in feature and feature['image'] is not None:
                    img = feature['image']
                    if not isinstance(img, Image.Image):
                        img = Image.fromarray(img)
                    feature['image'] = plant_augmentations(img)

        # Call parent collator
        return super().__call__(features)


train_collator = AugmentedUnslothVisionDataCollator(model, tokenizer, is_training=True)
eval_collator = AugmentedUnslothVisionDataCollator(model, tokenizer, is_training=False)


Unsloth: Model does not have a default image size - using 512
Unsloth: Model does not have a default image size - using 512


In [10]:
from unsloth.chat_templates import standardize_data_formats
train_dataset_c_s = standardize_data_formats(train_dataset_c)
val_dataset_c_s = standardize_data_formats(val_dataset_c)
print(tokenizer.apply_chat_template(train_dataset_c_s[10]["messages"]))


<bos><start_of_turn>user
<image_soft_token>As a botanist, observe the image of the flower and describe its visual features and species_name in JSON format. {color, inflorescencetype, inflorescence_description, flower_arrangement, flower_density, species, family, genus}<end_of_turn>
<start_of_turn>model
{"color": "Vibrant sky-blue petals fading to a white center, with prominent dark purple anthers.", "inflorescencetype": "Cyme", "inflorescence_description": "Loose, flat-topped terminal cymes with numerous small flowers opening sequentially.", "flower_arrangement": "Small, star-like flowers arranged in branching clusters at the apex of the stems.", "flower_density": "Moderately dense", "species": "Sedum caeruleum", "family": "Crassulaceae", "genus": "Sedum"}<end_of_turn>



We now have to apply the chat template for `Gemma-3` onto the conversations, and save it to `text`. We remove the `<bos>` token using removeprefix(`'<bos>'`) since we're finetuning. The Processor will add this token before training and the model expects only one.

Let's see how the chat template did! Notice there is no `<bos>` token as the processor tokenizer will be adding one.

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [11]:
from transformers import EarlyStoppingCallback
from trl import SFTTrainer, SFTConfig
from datetime import datetime;
run_name = "gemma-3n-botanist"+(datetime.now().strftime('_%y%m%d-%H%M'))

FastModel.for_training(model) # Enable for training!

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_c,
    eval_dataset=val_dataset_c,
    processing_class=tokenizer.tokenizer,
    data_collator=train_collator,
    eval_data_collator=eval_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
    args = SFTConfig(
        per_device_eval_batch_size = 24,
        per_device_train_batch_size = 24,
        gradient_accumulation_steps = 1,
        #gradient_checkpointing = False,
        #gradient_checkpointing_kwargs = {"use_reentrant": False},
        max_grad_norm = 0.5,              # max gradient norm based on QLoRA paper
        warmup_steps = 20,
        max_steps = -1,
        num_train_epochs = 4,          # Set this instead of max_steps for full training runs
        learning_rate = 3e-4,
        lr_scheduler_type = "cosine_with_min_lr",
        lr_scheduler_kwargs= { "min_lr": 1e-4 },
        logging_steps = 1,
        eval_steps = 200,
        save_steps = 200,
        eval_strategy = "steps",
        save_strategy="steps",
        optim = "adamw_torch_fused",
        weight_decay = 0.05,
        seed = 3407,
        output_dir = output_dir ,
        # ⚡ Key settings for GPU utilization
        dataloader_num_workers=8,
        dataloader_pin_memory=True,  # Faster GPU transfer
        dataloader_prefetch_factor=2,  # Prefetch batches
        bf16=True,
        report_to = "wandb",
        run_name = run_name,  # Name of the run in wandb

        load_best_model_at_end=True,
        metric_for_best_model='eval_loss',
        greater_is_better=False,


        # You MUST put the below items for vision finetuning:
        remove_unused_columns = False,
        dataset_text_field = "",
        dataset_kwargs = {"skip_prepare_dataset": True},
        #max_seq_length = 1024,
    )
)

In [ ]:
torch._dynamo.config.cache_size_limit = 256  # Default is 64, increase as needed
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 19,754 | Num Epochs = 4 | Total steps = 3,296
O^O/ \_/ \    Batch size per device = 24 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (24 x 1 x 1) = 24
 "-____-"     Trainable parameters = 614,727,680 of 8,464,705,872 (7.26% trained)
wandb: Currently logged in as: mekpro (mekproai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss
200,0.379600,0.589601
400,0.252800,0.491036
600,0.106300,0.290566
800,0.124800,0.199648
1000,0.087800,0.194192
1200,0.084900,0.186292
1400,0.053600,0.148489
1600,0.041500,0.131423


Unsloth: Will smartly offload gradients to save VRAM!


<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Gemma-3` team, the recommended settings for inference are `temperature = 1.0, top_p = 0.95, top_k = 64`

In [ ]:
# write a botanist test , give model image_url and prompt -> predict species , and , inflorencetype , validate species and inflorencetype. count score
# 1. prepare image_url from dataset upload to s3 public, map with list of species and inflorencetype , return list of (image_url, species, inflorencetype)
# llm_inference_tester,
# : read the csv (image_url, species, inflorescencetype) , https://mekpro.ap-south-1.linodeobjects.com/botanist/random_sample_100.csv
# test by inference each one, with image text, get json result, parse json check species and inflorescencetype , count correct species , count correct inflorescencetype
# image domain prefix is: https://mekpro.ap-south-1.linodeobjects.com/botanist/
# prompt is  "As a botanist, observe the image of the flower and describe its visual features and species_name in JSON format. {color, inflorescencetype, inflorescence_description, flower_arrangement, flower_density, unique_visual_description, morphological_traits_observable_in_photograph, species, family, genus,  visual_contrast_with_similar_species}"


# https://mekpro.ap-south-1.linodeobjects.com/botanist%2Ftrain%2F1355868%2F37975562c06628d9d3e38e03df6051c4b1c4f692.jpg


link = "https://raw.githubusercontent.com/plantnet/PlantNet-300K/refs/heads/main/images/1.jpg" # Cirsium arvense
#link = "https://mekpro.ap-south-1.linodeobjects.com/pelargonium_peltatum_4.jpg"
instruction = "As a botanist, observe the image of the flower and describe its visual features and species_name in JSON format. {color, inflorescencetype, inflorescence_description, flower_arrangement, flower_density, species, family, genus}"
FastModel.for_inference(model) # Enable for training!


messages = [{
    "role" : "user",
    "content": [
        { "type": "image", "image" : link },
        { "type": "text",  "text" : instruction }
    ]
}]

# You might have to wait 1 minute for Unsloth's auto compiler
do_gemma_3n_inference(messages, max_new_tokens = 512)

messages = [{
    "role" : "user",
    "content": [
        { "type": "text",  "text" : instruction }
    ]
}]

#do_gemma_3n_inference(messages, max_new_tokens = 512)

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
from huggingface_hub import HfApi,login
from google.colab import userdata
HF_TOKEN=userdata.get('HF_TOKEN')
login(token=HF_TOKEN)

#!huggingface-cli repo-files "mekpro/gemma-3n-e4b-gemma-3n-botanist-observe" delete "*"
#!huggingface-cli repo-files "mekpro/gemma-3n-e4b-gemma-3n-botanist-observe-merged" delete "*"
#!huggingface-cli repo-files "mekpro/gemma-3n-e4b-gemma-3n-botanist-observe-gguf" delete "*"

<a name="Save"></a>
### Saving models


In [ ]:
# model.save_pretrained("gemma-3n")  # Local saving
# tokenizer.save_pretrained("gemma-3n")
# model.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving
# tokenizer.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving
from huggingface_hub import HfApi,login
from google.colab import userdata
HF_TOKEN=userdata.get('HF_TOKEN')
login(token=HF_TOKEN)

import transformers
model_save_path = f"gemma-3n-botanist8"

model.save_pretrained(output_dir)  # Local saving
tokenizer.save_pretrained(output_dir)
model.save_pretrained_merged(output_dir+"-merged", tokenizer)
model.push_to_hub("mekpro/"+model_save_path, token = HF_TOKEN) # Online saving
tokenizer.push_to_hub("mekpro/"+model_save_path, token = HF_TOKEN) # Online saving


model.push_to_hub_merged(
    repo_id="mekpro/"+model_save_path+"-merged",
    tokenizer=tokenizer,
    save_method="merged_16bit",
    token = HF_TOKEN
)

#model.push_to_hub_gguf(model_save_path+"-gguf", repo_id="mekpro/gemma-3n-e4b-botanist-gguf-grpo", quantization_type="q8_0", token=HF_TOKEN)


For clear file (if needed)

staty connected



function ClickConnect() {
    console.log("Clicked on connect button");
    document.querySelector("colab-connect-button").shadowRoot.querySelector("#connect").click();
}
setInterval(ClickConnect, 60000);

monitoring:


nvidia-smi --query-gpu=power.draw,memory.used,memory.total,utilization.memory,utilization.gpu,clocks.current.graphics,clocks.current.sm,clocks.current.memory,clocks.current.video -l 1 --format=csv,noheader